In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
from scipy.stats import zscore

In [2]:
def extract_data(source):
    """
    Load multiple excel files into one pandas dataframe
    :param source: source path
    :return: dataframe
    """

    # read files
    path = Path(source)
    # create empty dataframe
    df = pd.DataFrame()
    # iterate all files
    for file in path.iterdir():
        df = df.append(pd.read_excel(file, engine="openpyxl"))

    return df

In [3]:
df = extract_data("../data/")
df.shape

In [ ]:
def transform_data(df, name):
    """
    filter only GBK data.
    :param df: main dataframe
    :param name: franchisee name
    :return dataframe
    """
    # # select only specific franchisee and franchisee activity
    # df = df.loc[df["Franchisee"].str.contains("GBK")]
    # rename columns to lowercase and add underscore between column names
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace(" ", "_")

    # sort values by date of transaction, user and transaction type
    df.sort_values(["date_of_transaction", "user", "transaction_type"], inplace=True)

    # select franchisee based on name
    df = df.loc[df["user"].str.contains(name) & df["user_type"].str.contains("Franchisee")]

    # drop unnecessary columns
    df.drop(columns=["transaction_sl", "user_id", "country", "user_estd_date", "transaction_type_level_2",
        "cost_name", "service", "attribute", "customer_name", "customer_id", "usd_net_amount", "currency_(iso_code)",
        "is_imported", "transaction_at", "exchange_rate"], inplace=True)
    df.reset_index(drop=True, inplace=True)

    # return df
    return df

In [ ]:
gbk = transform_data(df, "GBK")
gbk.shape

In [ ]:
def transactional_acitivity(df):
    """
    from the franchisee dataframe determine transactional activities
    :param df: franchisee dataframe
    :return dataframe
    """
    # return transactional activity
    # groupby date of transaction, transaction type, business category, product category & transaction id
    df = df.groupby(["date_of_transaction", "transaction_type", "business_category", "product_category",
            "transaction_id"], as_index=False) \
        .agg(unique_product = ("transaction_id", "count"),
            total_amount = ("net_amount", "sum"))
    
    # find daily number of transactions
    num_transaction = df.groupby("date_of_transaction", as_index=False)["transaction_id"].count()
    # convert date of transaction to datetime
    num_transaction["date_of_transaction"] = pd.to_datetime(num_transaction["date_of_transaction"])

    # groupby date of transaction, transaction type & product category & unstack the dataframe
    df = df.groupby(["date_of_transaction", "transaction_type"]) \
        .agg(total_amount = ("total_amount", "sum")).unstack()
    
    # convert multilevel column to single level & rename column names (list comprehension)
    df.columns = ["_".join(col) for col in df.columns]
    # convert all column names to lowercase
    df.columns = df.columns.str.lower()

    # fill missing dates with 0 for time series analysis
    # create idx based on start and end date of the franchisee activity
    idx = pd.date_range(df.index.min(), df.index.max())
    # set index as datetime index
    df.index = pd.DatetimeIndex(df.index)
    # reindex the dataframe
    df = df.reindex(idx, fill_value=np.nan)

    # reset_index
    df.reset_index(inplace=True)

    # rename index to date of transaction
    df.rename(columns={"index":"date_of_transaction"}, inplace=True)

    # add number of transaction column to the dataframe
    df = df.merge(num_transaction, on="date_of_transaction", how="left")

    # rename transaction id to number of transaction
    df.rename(columns={"transaction_id":"number_of_transaction"}, inplace=True)

    # replace NaN with 0
    df.fillna(0, inplace=True)

    # daily cash-flow
    df["cashout"] = df["total_amount_processing"] + df["total_amount_purchase"]
    df["cashin"] = df["total_amount_sale"]
    df.drop(columns=["total_amount_processing", "total_amount_purchase", "total_amount_sale"], inplace=True)
    
    return df

In [ ]:
gbk = transactional_acitivity(gbk)
gbk.head(30)

In [ ]:
if __name__ == "__main__":
    # extract data
    SOURCE_DATA = "../data/"
    df = extract_data(SOURCE_DATA)

    # transform data of a specific franchisee
    name = "GBK"
    gbk = transform_data(df, name)

    # transactional_activity
    gbk = transactional_acitivity(gbk)
    
    print(gbk.head())